In [ ]:
import json
from pathlib import Path
import pandas as pd

In [ ]:
def get_model_reports(model):
    report_paths = [f for f in Path(f'./{model}/report').iterdir()]

    result = {}
    for report_path in report_paths:
        with open(report_path, 'r') as fin:
            result[report_path.stem] = json.load(fin)
    
    return result

def get_reports():
     models = [str(f) for f in Path('./').iterdir() if f.is_dir()]

     result = {}
     for model in models:
         model_reports = get_model_reports(model)
         for report_name in model_reports:
             result[f'{model} {report_name}'] = model_reports[report_name]
    
     return result

In [ ]:
reports = get_reports()

## Common report

In [ ]:
pd.DataFrame(reports)